In [229]:
import pygame
import vectors
import numpy as np
from random import randint, uniform
from math import pi, cos, sin, sqrt

In [230]:
class PolygonModel():
    def __init__(self,points):
        self.points = points
        self.rotation_angle = 0
        self.x = 0
        self.y = 0
    def transformed(self):
        rotated = [vectors.rotate2d(self.rotation_angle, v) for v in self.points]
        return [vectors.add((self.x,self.y), v) for v in rotated]  
    def segments(self):
        point_count = len(self.points)
        points = self.transformed()
        return [(points[i], points[(i+1)%point_count])
                for i in range(0,point_count)]      
    def does_intersect(self, other_segment):
        for segment in self.segments():
            if do_segments_intersect(other_segment,segment):
                return True
        return False
    def does_collide(self, other_polygon):
        for other_segment in other_polygon.segments():
            if self.does_intersect(other_segment):
                return True
        return False
        
class Ship(PolygonModel):
    def __init__(self):
        super().__init__([(0.5,0), (-0.25,0.25), (-0.25,-0.25)])
    def laser_segment(self):
        dist = 20. * sqrt(2)
        x,y = self.transformed()[0]
        return((x,y), 
              (x + dist * cos(self.rotation_angle),
               y + dist * sin(self.rotation_angle)))

In [231]:
class Asteroid(PolygonModel):
    def __init__(self):
        sides = randint(5,9)
        vs = [vectors.to_cartesian((uniform(0.5,1), 2*pi*i/sides)) for i in range(0, sides)]
        super().__init__(vs)

In [232]:
ship = Ship()

In [233]:
asteroid_count = 10
asteroids = [Asteroid() for _ in range(0, asteroid_count)]

In [234]:
for ast in asteroids:
    ast.x = randint(-9,9)
    ast.y = randint(-9,9)

In [235]:
GREEN = (0, 255, 0)

In [236]:
def draw_poly(screen, polygon_model, color=GREEN):
    pixel_points = [to_pixels(x,y) for x,y in polygon_model.transformed()]
    pygame.draw.aalines(screen, color, True, pixel_points, 10)

In [237]:
# laser = ship.laser_segment()
# keys = pygame.key.get_pressed()
# if keys[pygame.K_SPACE]:
#     draw_segment(*laser)
#     for ast in asteroids:
#         if ast.does_intersect(laster):
#             asteroids.remove(ast)

In [238]:
width, height = 400, 400
def to_pixels(x,y):
    return (width/2 + width * x / 20, height/2 - height * y / 20)

In [239]:
def standard_form(v1,v2):
    x1, y1 = v1
    x2, y2 = v2
    a = y2-y1
    b = x1-x2
    c = x1*y2 - x2*y1
    return a,b,c

In [240]:
def intersection(u1,u2,v1,v2):
    a1, b1, c1 = standard_form(u1,u2)
    a2, b2, c2 = standard_form(v1,v2)
    m = np.array(((a1,b1), (a2,b2)))
    c = np.array((c1,c2))
    return np.linalg.solve(m,c)

In [241]:
def do_segments_intersect(s1,s2):
    u1,u2 = s1
    v1,v2 = s2
    d1, d2 = vectors.distance(*s1), vectors.distance(*s2)
    try:
        x,y = intersection(u1,u2,v1,v2)
        return (vectors.distance(u1, (x,y)) <= d1 and
                vectors.distance(u2, (x,y)) <= d1 and
                vectors.distance(v1, (x,y)) <= d2 and
                vectors.distance(v2, (x,y)) <= d2)
    except np.linalg.linalg.LinAlgErr:
        return False

In [242]:
do_segments_intersect(((1,2),(1, 0)), ((2,3),(3, -2)))

False

In [243]:
do_segments_intersect(((0,2),(1,-1)),((0,0),(4,0)))

True

In [244]:
points = [(2,7), (1,5), (2,3), (4,2), (6,2), (7,4), (6,6), (4,6)]

In [245]:
polygon = PolygonModel(points)

In [246]:
lzr = [(0,0),(7,7)]

In [247]:
polygon.does_intersect(lzr)

True

In [248]:
matrix = np.array(((1,1,-1),(0,2,-1),(1,0,1)))
vector = np.array((-1,3,2))
inverse = np.linalg.inv(matrix)

In [249]:
print(inverse)

[[ 0.66666667 -0.33333333  0.33333333]
 [-0.33333333  0.66666667  0.33333333]
 [-0.66666667  0.33333333  0.66666667]]


In [250]:
np.matmul(inverse,vector)

array([-1.,  3.,  3.])